## run this cell before starting spark cluster connection (corrects module error)

In [1]:
import os
print os.environ['PYSPARK_PYTHON']
os.environ['PYSPARK_PYTHON']=os.environ['LCG_VIEW'] + '/bin/python'
print os.environ['PYSPARK_PYTHON']

/etc/spark/python
/cvmfs/sft.cern.ch/lcg/views/LCG_93/x86_64-slc6-gcc62-opt/bin/python


### Import necessary spark and python stuff

In [2]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.functions import from_json
from pyspark.ml.feature import VectorAssembler

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

### select the data

In [4]:
df_ps_final = spark.read.json("/user/sprenner/training_official_april_hourly_proportion_prefetching.json")

In [5]:
#df_ps_final = df_ps_final.select('accountHash','clientStateHash','count','datasetHash','datasetScopeHash','dataset_versionHash','datatypeHash',
#                                 'eventTypeHash','eventVersionHash','filenameHash','filesize_double','hits','hostnameHash','hourHash','int_percentage',
#                                 'prod_stepHash','projectHash','protocolHash','remoteSiteHash','run_numberHash','scopeHash','stream_nameHash',
#                                 'traceIdHash','traceIpHash','traceTimeentryUnix','transferStart','uuidHash')

df_ps_final = df_ps_final.select('count','datatypeHash','eventTypeHash','filesize_double','hourHash','int_percentage','max_percentage','traceTimeentryUnix','transferStart')

In [6]:
df_ps_final.printSchema()

root
 |-- count: long (nullable = true)
 |-- datatypeHash: long (nullable = true)
 |-- eventTypeHash: long (nullable = true)
 |-- filesize_double: double (nullable = true)
 |-- hourHash: long (nullable = true)
 |-- int_percentage: long (nullable = true)
 |-- max_percentage: long (nullable = true)
 |-- traceTimeentryUnix: double (nullable = true)
 |-- transferStart: double (nullable = true)



In [7]:
df_ps_final.count()

777286

### Scale Numeric Features

In [8]:
df_ps_final.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_ps_final.columns]).limit(10).toPandas()

count  datatypeHash  eventTypeHash  filesize_double  hourHash  \
0   1624             0              0                0         0   

   int_percentage  max_percentage  traceTimeentryUnix  transferStart  
0          197902          111420                   0              0

In [9]:
df_ps_final = df_ps_final.fillna(0,['count','int_percentage', 'max_percentage'])

In [10]:
df_ps_final = df_ps_final.dropna()
df_ps_final.count()

777286

In [11]:
#df_ps_final.write.format("json").save("/user/sprenner/forward_selection_data.json")

In [12]:
pd.set_option('display.max_columns', None)
df_ps_final.describe().toPandas()

summary              count          datatypeHash eventTypeHash  \
0   count             777286                777286        777286   
1    mean  3673.703860612439   4.947060556405943E8    5.937808E7   
2  stddev  7419.355144319631  1.1700303292632785E9           0.0   
3     min                  0           -2070622869      59378080   
4     max             199996            2124512252      59378080   

       filesize_double               hourHash      int_percentage  \
0               777286                 777286              777286   
1   6.11402028756827E8  -2.5176970566176414E7  23.303134238877323   
2  1.240877180994579E9   1.2215909376898787E9  28.438942532079484   
3                304.0            -2138243719                   0   
4      1.1991446515E10             2144044799                 100   

       max_percentage    traceTimeentryUnix         transferStart  
0              777286                777286                777286  
1   47.19760937415572  1.5236944545257246E9  1.5236943924140284E9  
2  40.226311268628685     652927.2270245681     652914.6027100746  
3                   0   1.522533840149492E9   1.522527378084161E9  
4                 100   1.525125586283086E9   1.525125342411194E9

In [13]:
from pyspark.ml.regression import LinearRegression
import copy


# starts forward selection
def forwardSelection(data, features, target):
    featureList = copy.deepcopy(features)
    selected = []
    #keep track what the current error is to minimize it
    referenceScore = None
    while len(featureList) > 0:
        best = getBestFeature(data, featureList, selected, target, referenceScore)
        if best is not None:
            referenceScore = best[1]
            selected.append(best[0])
            featureList.remove(best[0])
            print("Selected " + str(best[0]))
            print("Current Feature Set: " + str(selected))
        else:
            print("No further improvement with removing features, stopping selection.")
            break
    return selected

# selects feature which yields the largest improvement together with already selected features (include)    
def getBestFeature(data, features, included, target, referenceScore):
    print("Getting next best feature...")
    if len(features) == 0:
        return "No features passed."
    include = copy.deepcopy(included)
    bestFeature = None
    for feature in features:
        # Don't include target feature, would be the best predictor otherwise
        if feature == target:
            print("Skipping " + str(feature))
            continue
        print("Calculating score for column " + str(feature) + " including " + str(include))
        # temporarily add current feature to feature set
        include.append(feature)
        assembler = VectorAssembler(inputCols=include, outputCol="features")
        output = assembler.transform(data)
        output = output.select(target, "features")
        train_data, test_data = output.randomSplit([.8,.2],seed=1234)
        error = regressionSquaredError(target, train_data)
        # first iteration
        if bestFeature is None:
            # first iteration overall
            if referenceScore is None:
                print("First feature overall found: " + str(feature) + " with score " + str(error))
                bestFeature = (feature, error)
                # for first iteration overall referenceScore must be set here
                referenceScore = error
            # first iteration but already reference error available
            elif error <= referenceScore:
                if error < referenceScore:
                    print("First feature for loop found: " + str(feature) + " with score " + str(error))
                    bestFeature = (feature, error)
                else:
                    print("First feature for loop found with same error: " + str(feature) + " with score " + str(error))
                    bestFeature = (feature, error)
        # the error must be smaller than previous selection and smaller than with fewer features
        elif error <= bestFeature[1] and error <= referenceScore:
            if error < bestFeature[1] and error < referenceScore:
                bestFeature = (feature, error)
                print("Overall better feature found: " + str(feature) + " with score " + str(error))
            elif error < bestFeature[1] and error == referenceScore:
                bestFeature = (feature, error)
                print("Better feature found with same error as reference: " + str(feature) + " with score " + str(error))
            elif error == bestFeature[1] and error < referenceScore:
                bestFeature = (feature, error)
                print("Better feature found with same error as current best feature: " + str(feature) + " with score " + str(error))
            elif error == bestFeature[1] and error == referenceScore:
                bestFeature = (feature, error)
                print("Feature with no effect found: " + str(feature) + " with score " + str(error))

        # remove temporary feature from feature set to start again with next feature
        include.remove(feature)
    return bestFeature
        
# performs linear regression and returns mean squared error      
def regressionSquaredError(label, data):
    lr = LinearRegression(labelCol=label, maxIter=10, regParam=0.3, elasticNetParam=0.8)
    linearModel = lr.fit(data)
    return linearModel.summary.rootMeanSquaredError

# performs linear regression and returns r2        
def regressionR2(label, data):
    lr = LinearRegression(labelCol=label, maxIter=10, regParam=0.3, elasticNetParam=0.8)
    linearModel = lr.fit(data)
    return linearModel.summary.r2

In [14]:
df_ps_final.schema.names

['count',
 'datatypeHash',
 'eventTypeHash',
 'filesize_double',
 'hourHash',
 'int_percentage',
 'max_percentage',
 'traceTimeentryUnix',
 'transferStart']

### Perform forward selection

In [15]:
selected_features = forwardSelection(df_ps_final, df_ps_final.schema.names, "max_percentage")

Getting next best feature...
Calculating score for column count including []
First feature overall found: count with score 36.6607006876
Calculating score for column datatypeHash including []
Overall better feature found: datatypeHash with score 36.4308674812
Calculating score for column eventTypeHash including []
Calculating score for column filesize_double including []
Calculating score for column hourHash including []
Calculating score for column int_percentage including []
Overall better feature found: int_percentage with score 27.4826884285
Skipping max_percentage
Calculating score for column traceTimeentryUnix including []
Calculating score for column transferStart including []
Selected int_percentage
Current Feature Set: ['int_percentage']
Getting next best feature...
Calculating score for column count including ['int_percentage']
First feature for loop found: count with score 26.1471987945
Calculating score for column datatypeHash including ['int_percentage']
Calculating score 

In [16]:
selected_features

['int_percentage',
 'count',
 'filesize_double',
 'datatypeHash',
 'traceTimeentryUnix',
 'hourHash',
 'eventTypeHash']